In [35]:
%run /home/dbaciur/NTU/NTU/notes/Constants.ipynb
%run /home/dbaciur/NTU/NTU/notes/CommonUtils.ipynb

Constants loaded


In [5]:
import torch
import torchvision as tv

import os
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import numpy as np
import intervals as I
import moviepy
from pathlib import Path
from moviepy.editor import VideoFileClip
import pandas as pd
import json as j
import pickle


### Load events & frames

In [3]:
events_df = pd.read_csv(C.VISUAL_EVENTS_PATH, converters=C.A_CONVERTERS)
events_df.head(1)

,Id,Unnamed: 0,Behavior,Behavior type,Behavioral category,Comment start,Comment stop,Duration (s),FPS,Media file,...,Total length,lecture_id,research_assistant_id,subject_id,BehaviorId,Video,Time,Start H (s),Stop H (s),Time H
0,1,1,Writing on slides,STATE,Providing clear explanation; Arousing interest,0.0,0,2739.504,29.83,C:/Users/ACER/Downloads/PH1011-PHYSICS_2015092...,...,7197.58,J1S1C11L1,RA13,J1S1C11,B_WRITING_SLIDES,PH1011-PHYSICS_20150922,1502.103,00:02:12:351,00:47:51:855,00:25:02:103


In [6]:
def unpickle_obj(path):
    with open(path, 'rb') as handle:
        return pickle.load(handle)

In [7]:
# this file is prepared separately in Python part of the project
event_ids_for_new_frames = unpickle_obj("/tmp/event_ids_for_new_chart_image_label_0_frames.pickle")
event_ids_for_new_frames

[90,
 96,
 99,
 100,
 105,
 106,
 107,
 110,
 111,
 112,
 113,
 119,
 122,
 170,
 214,
 217,
 218,
 219,
 220,
 223,
 224,
 225,
 226,
 229,
 230,
 231,
 232,
 233,
 234,
 235,
 236,
 237,
 238,
 241,
 242,
 243,
 244,
 249,
 250,
 251,
 252,
 253,
 288,
 346,
 347,
 348,
 354,
 355,
 360,
 361,
 364,
 367,
 368,
 369,
 372,
 377,
 378,
 419,
 475,
 476,
 477,
 478,
 479,
 480,
 485,
 488,
 566,
 636,
 637,
 684,
 729,
 738,
 751,
 752,
 763,
 764,
 765,
 766,
 767,
 768,
 770,
 824,
 825,
 826,
 827,
 828,
 832,
 835,
 836,
 838,
 841,
 842,
 843,
 844,
 846,
 847,
 849,
 851,
 852,
 853,
 854,
 855,
 856,
 858,
 860,
 861,
 863,
 865,
 868,
 870,
 871,
 872,
 874,
 876,
 877,
 878,
 881,
 884,
 885,
 886,
 887,
 888,
 889,
 891,
 893,
 895,
 896,
 897,
 898,
 899,
 900,
 901,
 902,
 903,
 904,
 906,
 908,
 909,
 910,
 912,
 915,
 916,
 919,
 921,
 923,
 949,
 950,
 951,
 955,
 956,
 957,
 958,
 960,
 961,
 962,
 963,
 964,
 965,
 966,
 967,
 968,
 971,
 972,
 974,
 976,
 977,
 978,
 

In [21]:
# Prepare info about layouts
video_durations = {}

for video in tqdm(events_df[C.A_VIDEO].unique()):
        
    video_path = f"{C.ORIGINAL_VIDEOS_DIR}/{video}.mp4"
    video_clip = VideoFileClip(video_path)
    
    video_durations[video] = video_clip.duration

In [22]:
video_layout_annotations = {}

for video in tqdm(events_df[C.A_VIDEO].unique()):
    
    if not os.path.exists(f"{C.LAYOUT_ANNOTATIONS_DIR}/{video}.pt"):
        print("omitting")
        continue
        
    layouts_torch = torch.load(f"{C.LAYOUT_ANNOTATIONS_DIR}/{video}.pt")
    video_layout_annotations[video] = layouts_torch.numpy()

omitting
omitting
omitting
omitting
omitting



In [28]:
def get_timestmap_layout(video, time):
    
    if video not in video_layout_annotations:
        print("omitting cause no layout info")
        return None
    layout_annotations = video_layout_annotations[video]
    layout_annotations_size = len(layout_annotations)
    
    video_duration = video_durations[video]
    layout_index_approximation = (time / video_duration) * layout_annotations_size
    layout_index_approximation = int(layout_index_approximation)
    
    if layout_index_approximation >= len(layout_annotations):
        print("omitting cause layout annotations cut too early")
        return None
    layout = layout_annotations[layout_index_approximation]
    layout = int(layout)
    
    return layout

def is_screen_layout(layout):
    return layout in [2, 4, 5, 6, 8, 9, 10]

def get_split_layouts(layout):
    # layout < 3 => whole screen is either Camera or Screen
    if layout < 3:
        layouts = [layout, layout]
        
    # layout < 7 => layout is divided into 3 parts:
    # - 1 vertical line in 1/3 of width middle that splits screen to L and R
    # - 1 horizontal line in the middle of L
    elif layout < 7:
        layouts = [[1,1], [1,2], [2,1], [2,2]][layout - 3]
    # layout >=7 => layout is split in half
    else:
        layouts = [[1,1], [1,2], [2,1], [2,2]][layout - 7]
    
    return layouts

def get_splits(frame_id, layout):
    if layout < 3:
        return [f"{frame_id}__Full", f"{frame_id}__Full"]
    
    return [f"{frame_id}__Left", f"{frame_id}__Right"]

### Create frames

In [31]:
frames_added = 0
frames_omitted_by_layout = 0
video_timestamps_omitted_by_layout = []

df_data = []
    
events_of_interest = events_df[events_df[C.A_EVENT_ID].isin(event_ids_for_new_frames)]
    
for _, event in events_of_interest.iterrows():

    video = event[C.A_VIDEO]
    start = event[C.A_START]
    stop = event[C.A_STOP]
    time = (start + stop) / 2
    event_id = event[C.A_EVENT_ID]
    beh_id = event[C.A_BEH_ID]
    ra_id = event[C.A_RA_ID]


    frame_id = f"{video}__{time}__{event_id}__{beh_id}__{ra_id}"

    layout = get_timestmap_layout(video, time)
    if layout is None:

        continue
        
    layouts = get_split_layouts(layout)
    splits = get_splits(frame_id, layout)

    if not is_screen_layout(layout):
        frames_omitted_by_layout = frames_omitted_by_layout + 1
        video_timestamps_omitted_by_layout.append((video, format_seconds(time)))
#                 print(f"bad layout {time_candidate}, {layout}")
        continue

    frames_added = frames_added + 1
    df_data.append([
        frame_id,
        video,
        time,
        event_id,
        beh_id,
        ra_id,
        [], # collision events = deliberately empty for now
        [], # behs = deliberately empty for now
        layout,
        layouts,
        splits,
        "new_frames_for_chart_image_label_0_gap_10"
    ])
            
print(frames_added)
print(frames_omitted_by_layout)

omitting cause no layout info
omitting cause no layout info
omitting cause no layout info
omitting cause no layout info
omitting cause no layout info
omitting cause no layout info
omitting cause no layout info
omitting cause no layout info
omitting cause no layout info
omitting cause no layout info
omitting cause no layout info
omitting cause no layout info
omitting cause no layout info
omitting cause no layout info
omitting cause no layout info
omitting cause no layout info
omitting cause no layout info
omitting cause no layout info
omitting cause no layout info
omitting cause no layout info
omitting cause no layout info
omitting cause no layout info
omitting cause no layout info
omitting cause no layout info
omitting cause no layout info
omitting cause no layout info
omitting cause no layout info
omitting cause no layout info
omitting cause no layout info
omitting cause no layout info
omitting cause no layout info
omitting cause no layout info
omitting cause no layout info
omitting c

In [48]:
video_timestamps_omitted_by_layout

[('16S1-PH1801-LEC_20161020', '00:24:33:022'),
 ('16S1-PH1801-LEC_20161020', '00:24:38:022'),
 ('16S1-PH1801-LEC_20161020', '00:24:43:022'),
 ('16S1-PH1801-LEC_20161020', '00:24:48:022'),
 ('16S1-PH1801-LEC_20161020', '00:24:53:022'),
 ('16S1-PH1801-LEC_20161020', '00:24:58:022'),
 ('16S1-PH1801-LEC_20161020', '00:25:03:022'),
 ('16S1-PH1801-LEC_20161020', '00:25:08:022'),
 ('16S1-PH1801-LEC_20161020', '00:25:13:022'),
 ('16S1-PH1801-LEC_20161020', '00:25:18:022'),
 ('16S1-PH1801-LEC_20161020', '00:25:23:022'),
 ('16S1-PH1801-LEC_20161020', '00:25:28:022'),
 ('16S1-PH1801-LEC_20161020', '00:25:33:022'),
 ('16S1-PH1801-LEC_20161020', '00:25:38:022'),
 ('16S1-PH1801-LEC_20161020', '00:25:43:022'),
 ('16S1-PH1801-LEC_20161020', '00:25:48:022'),
 ('16S1-PH1801-LEC_20161020', '00:25:53:022'),
 ('16S1-PH1801-LEC_20161020', '00:25:58:022'),
 ('16S1-PH1801-LEC_20161020', '00:26:03:022'),
 ('16S1-PH1801-LEC_20161020', '00:26:08:022'),
 ('16S1-PH1801-LEC_20161020', '00:26:13:022'),
 ('16S1-PH180

In [32]:
new_frames_df = pd.DataFrame(
        data=df_data,
        columns=[
            C.F_ID,
            C.F_VIDEO,
            C.F_TIME,
            C.F_EVENT_ID,
            C.F_BEH_ID,
            C.F_RA_ID,
            C.F_COLL_EVENTS,
            C.F_BEHS,
            C.F_LAYOUT,
            C.F_LAYOUTS,
            C.F_SPLITS,
            C.F_ORIGIN,
        ]
    )

In [33]:
new_frames_df.head()

,Id,Video,Time,Event id,Beh id,RA,Collision events,Behs,Layout,Layouts,Splits,Origin
0,PH1104-MECHANICS_20150817_PC1__2886.1575000000...,PH1104-MECHANICS_20150817_PC1,2886.1575,729,B_WRITING_BOARD,RA13,[],[],4,"[1, 2]",[PH1104-MECHANICS_20150817_PC1__2886.157500000...,new_frames_for_chart_image_label_0_gap_10
1,PH1104-MECHANICS_20150817_PC1__44.2295__738__B...,PH1104-MECHANICS_20150817_PC1,44.2295,738,B_MOVEMENT,RA2,[],[],5,"[2, 1]",[PH1104-MECHANICS_20150817_PC1__44.2295__738__...,new_frames_for_chart_image_label_0_gap_10
2,PH1104-MECHANICS_20150817_PC1__1881.4025__751_...,PH1104-MECHANICS_20150817_PC1,1881.4025,751,B_STANDS,RA2,[],[],5,"[2, 1]",[PH1104-MECHANICS_20150817_PC1__1881.4025__751...,new_frames_for_chart_image_label_0_gap_10
3,PH1104-MECHANICS_20150817_PC1__1967.781__752__...,PH1104-MECHANICS_20150817_PC1,1967.7810,752,B_STANDS,RA2,[],[],4,"[1, 2]",[PH1104-MECHANICS_20150817_PC1__1967.781__752_...,new_frames_for_chart_image_label_0_gap_10
4,PH1104-MECHANICS_20150817_PC1__2687.4440000000...,PH1104-MECHANICS_20150817_PC1,2687.4440,763,B_WRITING_BOARD,RA2,[],[],4,"[1, 2]",[PH1104-MECHANICS_20150817_PC1__2687.444000000...,new_frames_for_chart_image_label_0_gap_10


In [34]:
len(new_frames_df)

3636

In [36]:
new_frames_df.to_csv(C.Frames.CHART_IMAGE_S10_LABEL_0, index=False)